In [ ]:
// run this cell to prevent Jupyter from displaying the null output cell
com.twosigma.beakerx.kernel.Kernel.showNullExecutionResult = false;

We need the `Counter` and `StoppingCounter` classes in this notebook. Run the next two cells to compile the classes.

In [ ]:
public class Counter {

    protected int value; 
    
    public Counter() {
        this.value = 0;
    }
    
    public Counter(int value) {
        this.value = value;
    }
    
    public Counter(Counter other) {
        this.value = other.value;
    }
    
    public int value() {
        return this.value;
    }
    
    public void advance() {
        if (this.value != Integer.MAX_VALUE) {
            this.value++;
        }
        else {
            this.value = 0;
        }
    }
    
    @Override
    public boolean equals(Object obj) {
        if (this == obj) {
            return true;
        }
        if (!(obj instanceof Counter)) {
            return false;
        }
        Counter other = (Counter) obj;
        if (this.value == other.value) {
            return true;
        }
        return false;
    }
    
    @Override
    public int hashCode() {
        int result = Integer.hashCode(this.value);

        return result;
    }
    
    @Override
    public String toString() {
        return "count: " + this.value;
    }
}

In [ ]:
public class StoppingCounter extends Counter {
    // no fields!
    
    public StoppingCounter() {
        this(0);
    }
    
    public StoppingCounter(int value) {
        super(value);
    }
    
    @Override
    public void advance() {
        if (this.value != Integer.MAX_VALUE) {
            this.value++;
        }
    }
}

<a id="notebook_id"></a>
# Polymorphism

One of the benefits of inheritance is that it allows the programmer to use different types of objects (that are related by inheritance) in the same way. For example, we can create a list of `Counter` references, use the list to store references to objects of any subclass of `Counter`, and then iterate over the elements of the list calling the `advance` method for each counter.

The ability for the same code to be used with different types of objects is called *polymorphism*. The Oxford dictionary definition of polymorphism is "the condition of occurring in several different forms". In Java, polymorphism is made possible by the fact that an object in an inheritance hierarchy has more than one type; namely, the object's types include its own class type, all of its superclass types, and all of its interface types.


## Declared types

In Java, a reference variable has what is called a *declared type*. For example:

```java
Counter c;
```

creates a variable `c` whose declared type is `Counter`. The declared type is the type that appears in front of a variable, field, or parameter.

What values can we store in the variable `c`? We can store any of the following values:

* `null`
* a reference to a `Counter` object
* a reference to any object having a type that is a subclass of `Counter`

For example, all of the following are legal:

In [ ]:
Counter c1 = null;
Counter c2 = new Counter();            // ok, Counter is a Counter
Counter c3 = new StoppingCounter();    // ok, StoppingCounter is a subclass of Counter
Counter c4 = c3;                       // ok, declared type of c3 is Counter

The example above is legal because of polymorphism: A subclass has multiple types including the types of all of its superclasses. Because a subclass object is usable whenever a superclass object is needed, we say that a subclass is *substitutable for* its superclass. Substitutability is discussed in greater detail in a following notebook.

Because `Object` is a superclass of all Java classes, it is legal to store any reference value in a variable whose declared type is `Object`:

In [ ]:
import java.util.ArrayList;

Object obj = new Object();
obj = "this is a string";           // ok, String is a subclass of Object
obj = new Counter();                // ok, Counter is a subclass of Object
obj = new ArrayList<Integer>();     // ok, ArrayList is a subclass of Object

Note that a superclass object is never substitutable for a subclass object. For example, the following statement will not compile:

In [ ]:
String s = new Object();

**Exercise 1** What is the declared type of each variable below?:

```java
Object obj;
String s;
List<String> t;
ArrayList<String> t2;
Counter c;
```

### The declared type determines what can be done with the variable, field, or parameter

The compiler uses the declared type of a variable, field, or parameter to determine if an operation involving the variable is legal. For example, suppose that we declare a variable of type `Object` and store a reference to an `ArrayList` in the variable:

In [ ]:
import java.util.ArrayList;

Object obj = new ArrayList<Integer>();

Because the declared type of `obj` is `Object` the only methods that can be called using `obj` are the methods defined in the class `Object`; in other words, the methods that we can call with a variable are the methods defined in the declared type. For example, we can `toString` using `obj` because `toString` is defined in `Object`:

In [ ]:
import java.util.ArrayList;

Object obj = new Object();
System.out.println(obj.toString());

Object obj1 = new ArrayList<Integer>();
System.out.println(obj1.toString());

Object obj2 = new StoppingCounter(1);      // a StoppingCounter whose value is 1
System.out.println(obj2.toString());

The `Object` version of `toString` returns a string starting with `java.lang.Object@` but notice that the strings printed using `obj1` and `obj2` are the strings corresponding to an empty list and for a counter having a value of 1. In other words, even though the types of `obj1` and `obj2` were declared as `Object` the method `toString` behaves as though `obj1` is a list and `obj2` is a counter. Keep this in mind when you read the next section about *Runtime types*.

If we call an `ArrayList` method using `obj1` the compiler will issue an error:

In [ ]:
import java.util.ArrayList;

Object obj1 = new ArrayList<Integer>();
obj1.add(1);                               // try to add an element to the list?

The call to `add` fails because the declared type `Object` has no method named `add`.

Similarly, if call a `Counter` method using `obj2` the compiler will issue an error:

In [ ]:
Object obj2 = new StoppingCounter(1);      // a StoppingCounter whose value is 1
System.out.println(obj2.value());

The call to `value` fails because the declared type `Object` has no method named `value`.

**Exercise 2** Does the following code cause the `Counter` or `StoppingCounter` version of `advance` to be called?

In [ ]:
// Exercise 2
Counter c = new StoppingCounter(Integer.MAX_VALUE);
c.advance();

## Runtime types

In an assignment expression, the type of the value on the right-hand side of the expression is the *runtime* type or *actual* type. For example, in the expression

```java
Object obj = "this is a string";
```

the runtime type of the object is `String`. The `Object` method `getClass` returns a reference to an object that represents the runtime class of a reference.

In [ ]:
Object obj = "this is a string";
System.out.println(obj.getClass());

### The runtime type determines which version of a method is called

The runtime type of an object determines which version of a method is called. Recall the example from the previous section:

In [ ]:
import java.util.ArrayList;

Object obj1 = new ArrayList<Integer>();
System.out.println(obj1.toString());

Object obj2 = new StoppingCounter(1);      // a StoppingCounter whose value is 1
System.out.println(obj2.toString());

The runtime type of the object referred to by `obj1` is `ArrayList`. The expression `obj1.toString()` uses the runtime type of `obj1` to determine which version of `toString` is called. In this case, the version of `toString` inherited by `ArrayList` is called. Similarly, using `obj2` to call `toString` results in the `StoppingCounter` version of `toString` to be called.

To expand on this example, consider a method that simply prints the string representation of any object (run the following cell to compile the class):

In [ ]:
public class Printer {
    public static void print(Object obj) {
        System.out.println(obj.toString());
    }
}

When we call the `print` method, the runtime type of the argument is used to determine which version of `toString` is called:

In [ ]:
import java.util.ArrayList;
import java.util.Date;

String s = "hi there";
Printer.print(s);

ArrayList<Integer> t = new ArrayList<>();
t.add(9);
t.add(1);
t.add(1);
Printer.print(t);

Object o = new Date();
Printer.print(o);

When the above cell is run:

* `Printer.print(s)` uses the `String` version of `toString` to be called because the runtime type of `s` is `String`
* `Printer.print(t)` uses the `ArrayList` version of `toString` to be called because the runtime type of `t` is `ArrayList`
* `Printer.print(o)` uses the `String` version of `Date` to be called because the runtime type of `o` is `Date`

We say that the `Printer` method `print` is polymorphic because its output changes depending on what (runtime) type is used as an argument to the method.

The method `print` does not know which version of `toString` to call until it is actually run. When the method is run, it looks at the runtime type of the parameter `obj` to determine which version of `toString` to call. This concept of using the runtime type to determine which version of a method to call is called *dynamic dispatch* or *late binding*. The words *dynamic* and *late* are used because the process occurs at runtime instead of at compile-time.

**Exercise 3** Is the `equals` method polymorphic?

**Exercise 4** Look up the documentation for the `AbstractCollection` method `contains`. This is the method that is inherited by collections such as `ArrayList` and `TreeSet` which determines if an element is in a collection. Explain why `contains` is a polymorphic method.

**Exercise 5** Look up the documentation for the `AbstractMap` method `get`. This is the method that is inherited by maps such as `HashMap` and `TreeMap` which returns a value corresponding to a key from a map. Explain why `get` is a polymorphic method.

**Exercise 6** Look up the documentation for the class `Collections`. Roughly what fraction of the methods are polymorphic?